In [1]:
import pandas as pd
from tqdm import tqdm
from scipy.spatial import cKDTree  
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiPoint, Polygon, shape, LineString

import json
import requests
import warnings
warnings.filterwarnings('ignore')

import os

import sp_functions as utc

In [6]:
#load data
all_bldg = gpd.read_file('all_bldg_in_target_area.geojson')
target = gpd.read_file('education_poi.geojson')

In [8]:
category = 'EDUCATION'
radius = 100
api_id = 'devportal-demo-20180625'
api_code = '9v2BkviRwi9Ot26kp2IysQ'

In [ ]:
result = utc.classify_bldg(all_bldg, target, category, radius, api_id, api_code)

In [25]:
result.head()

,UID,ADDRESS,category,restriction
0,ea3e66d3-0307-41f4-8a78-5f113a7314fa,"город Москва, 2-я улица Машиностроения, дом 17...",HEALTH,green
1,b7132015-fa80-48c3-985d-bd8b6c62ca27,"город Москва, Пятницкая улица, дом 66, строение 2",HEALTH,green
3,993426c3-167e-4cbf-80f4-88495030f715,"город Москва, Новокузнецкая улица, дом 38, стр...",HEALTH,green
4,24f38bff-ea3a-498b-9f67-33bc6d3e080d,"город Москва, улица Малая Ордынка, дом 18, стр...",HEALTH,green
5,417fae86-c6ae-4074-99bf-78f18ff57084,"город Москва, Летниковская улица, дом 16",HEALTH,green


In [26]:
#export result
result.to_csv('{}_result.csv'.format(category), index=False)

In [27]:
#load all new results
education = pd.read_csv('EDUCATION_result.csv')
health = pd.read_csv('HEALTH_result.csv')
subway = pd.read_csv('SUBWAY_EXIT_result.csv')
ot= pd.read_csv('BUS_STOP_result.csv')


#get final classification
part1 = pd.merge(education, health, on = ['UID', 'ADDRESS'], how='inner')
part2 = pd.merge(subway, ot, on = ['UID', 'ADDRESS'], how='inner')

all_parts = pd.merge(part1, part2, on=['UID', 'ADDRESS'], how='inner')

all_parts = all_parts.rename(columns={'category_x_x':'category_1',
                                      'category_y_x':'category_2',
                                      'category_x_y':'category_3',
                                      'category_y_y':'category_4',
                                      'restriction_x_x':'restriction_1',
                                      'restriction_y_x':'restriction_2',
                                      'restriction_x_y':'restriction_3',
                                      'restriction_y_y':'restriction_4'})

onlyred = all_parts[(all_parts['restriction_1']=='red')|
                    (all_parts['restriction_2']=='red')|
                    (all_parts['restriction_3']=='red')|
                    (all_parts['restriction_4']=='red')]

only_green = all_parts[(all_parts['restriction_1']=='green')&
                    (all_parts['restriction_2']=='green')&
                    (all_parts['restriction_3']=='green')&
                    (all_parts['restriction_4']=='green')]

only_yellow = all_parts[~all_parts['UID'].isin(onlyred['UID'])]
only_yellow = only_yellow[~only_yellow['UID'].isin(only_green['UID'])]

onlyred['final_restriction'] = 'red'
only_yellow['final_restriction'] = 'yellow'
only_green['final_restriction'] = 'green'

final = pd.concat([onlyred, only_yellow, only_green])

final.head()

,UID,ADDRESS,category_1,restriction_1,category_2,restriction_2,category_3,restriction_3,category_4,restriction_4,final_restriction
259,f0c1e967-a865-4ed2-a81c-2e1db883b091,"город Москва, Большая Серпуховская улица, дом ...",EDUCATION,green,HEALTH,green,SUBWAY_EXIT,red,BUS_STOP,red,red
484,e872c60a-d210-45b0-b107-47463e71324b,"город Москва, Волгоградский проспект, дом 32, ...",EDUCATION,green,HEALTH,green,SUBWAY_EXIT,red,BUS_STOP,red,red
524,c2eae677-e6e4-4532-97e7-e501e0cea9ea,"город Москва, Большая Серпуховская улица, дом ...",EDUCATION,green,HEALTH,green,SUBWAY_EXIT,red,BUS_STOP,red,red
722,ef2928ae-6c95-4741-9c3d-3fd0e0bfcbe1,"город Москва, Шарикоподшипниковская улица, дом...",EDUCATION,green,HEALTH,green,SUBWAY_EXIT,red,BUS_STOP,red,red
877,856a1c90-1059-427b-82a9-9e1da3d58310,"город Москва, улица Зацепский Вал, дом 14",EDUCATION,green,HEALTH,green,SUBWAY_EXIT,red,BUS_STOP,red,red


In [28]:
final.to_csv('final_building_restriction.csv', index=False)